<a href="https://colab.research.google.com/github/l2yao/InfiniteLife/blob/main/Youtube_SRT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installing Whisper

The commands below will install the Python packages needed to use Whisper models and evaluate the transcription results.

In [ ]:
! pip install git+https://github.com/openai/whisper.git
! pip install git+https://github.com/nficano/pytube

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-o7j47gac
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-o7j47gac
  Resolved https://github.com/openai/whisper.git to commit c09a7ae299c4c34c5839a76380ae407e7d785914
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 38.8 MB/s eta 0:00:00
  Created wheel for openai-whisper: filename=openai_whisper-20230314-py3-none-any.whl size=798038 sha256=8dd5fd9d0aa2fa74a43e4695d4d2b2e8a26b530552dfb61f8932efd24f0bf367
  Stored in directory: /tmp/pip-ephem-wheel-cache-i_21wd24/wheels/fe/03/29/e7919208d11b4ab32972cb448bb84a9a675d92cd52c9a48341
Successfully built openai-whisper
Looking in indexes: https://pypi.org/simple, https://us-p

In [ ]:
import io
import os
import numpy as np

try:
    import tensorflow  # required in Colab to avoid protobuf compatibility issues
except ImportError:
    pass

import torch
import pandas as pd
import urllib
import tarfile
import whisper



DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# Running inference on the dataset using a medium Whisper model

The following will take a few minutes to transcribe and translate utterances in the dataset.

In [ ]:
model = whisper.load_model("medium")
print(
    f"Model is {'multilingual' if model.is_multilingual else 'English-only'} "
    f"and has {sum(np.prod(p.shape) for p in model.parameters()):,} parameters."
)

100%|██████████████████████████████████████| 1.42G/1.42G [00:14<00:00, 106MiB/s]


Model is multilingual and has 762,321,920 parameters.


In [ ]:
from pytube import YouTube

youtube_url = "https://www.youtube.com/watch?v=HrP1qo1NUoI" #@param {type:"string"}
yt = YouTube(youtube_url)
video = yt.streams.filter(only_audio=True).first()
out_file=video.download()
base, ext = os.path.splitext(out_file)
new_file = base+'.mp3'
os.rename(out_file, new_file)

In [ ]:
transcription = model.transcribe(new_file)

In [ ]:
from whisper.utils import get_writer

writer = get_writer("srt", ".")
options = {"max_line_width": 1000, "max_line_count": 1000, "highlight_words": False}
writer(transcription, new_file, options)